In [7]:
from database import database
from matplotlib import pyplot as plt
import sklearn.feature_extraction.image
from sklearn.feature_extraction.image import extract_patches
import tables
import random
from SRCNN import run
from build_model import build_model
import tensorflow as tf

In [11]:
random.seed(31415926)
args = {}
args['filedir'] = 'train_data'
args['pattern'] = '*.bmp'
args['database_name'] = 'mock'
args['export_dir'] = 'export'
args['patch_shape'] = (32,32,3)
args['stride_size'] = 16



data_set = database(**args)
data_set.initialize()

100%|██████████| 10/10 [00:00<00:00, 48.11it/s]


{'img': 17202, 'label': 17202}

In [12]:
#run to train - use dataset['train',idx] to slice the chunkized hdf5 arrays
run(data_set, num_epoch = 15, batch_size = 32)

Preparing data...


  1%|          | 4/496 [00:00<00:14, 33.71it/s]

EPOCH: 0 Avg Loss: 0.16795728552801115


  2%|▏         | 8/496 [00:00<00:09, 53.07it/s]

EPOCH: 1 Avg Loss: 0.06442079504936814


...

EPOCH: 12 Avg Loss: 0.03253633381457128


  2%|▏         | 11/496 [00:00<00:04, 105.21it/s]

EPOCH: 13 Avg Loss: 0.03145375487304503


100%|██████████| 496/496 [00:05<00:00, 92.05it/s] 

EPOCH: 14 Avg Loss: 0.030415961475697376
